Teste Prático para Engenheiros de Dados - Itaú RTDA

In [ ]:
# Importa libs necessárias

import pandas as pd
import boto3
import io

from pyspark.sql import SparkSession, functions as f
from pyspark.sql.types import StructType, StructField, StringType, DateType, FloatType

from dotenv import load_dotenv
from os import getenv

import requests as req
import json

load_dotenv()


1. ETL e Manipulação de Dados

Utilize o arquivo sales_data.csv e:
Limpe os dados removendo linhas duplicadas e tratando valores ausentes.
Transforme o valor da venda de uma moeda fictícia para USD usando a taxa de conversão de 1 FICT = 0.75 USD.
Carregue os dados limpos e transformados em um banco de dados relacional.

In [ ]:

# Implementação estrutural

# Passo 1: Extração e Limpeza dos Dados
# Carrega o arquivo CSV
df_sales = pd.read_csv('input/sales_data.csv')

# Remove linhas duplicadas
df_sales = df_sales.drop_duplicates()

# Tratar valores ausentes (caso existam)
# Substitui valores ausentes por zero.
df_sales = df_sales.fillna(0)

# Passo 2: Transformação dos Dados
# Converter o valor da venda de moeda fictícia (FICT) para USD usando a taxa de conversão de 1 FICT = 0.75 USD
df_sales['usd_sale_value'] = df_sales['sale_value'] * 0.75

df_sales.to_csv("output/struct_sales_data.csv", index=False)

### 2. Análise com Apache Spark(utilize PySpark ou Spark)

#### Descrição

Dado um conjunto fictício de logs `website_logs.csv`:

- Identifique as 10 páginas mais visitadas.
- Calcule a média de duração das sessões dos usuários.
- Determine quantos usuários retornam ao site mais de uma vez por semana.

In [ ]:
# Cria sessão Spark

spark = (SparkSession.builder
                     .master("local[1]")
                     .getOrCreate()
         )

In [ ]:
# Importa o Dataframe
schema = StructType(
    [
          StructField("user_id", StringType())
         ,StructField("page_url", StringType())
         ,StructField("session_duration", FloatType())
         ,StructField("date", DateType())
    ]
)

df_website_access = spark.read.csv("input/website_logs.csv", header=True, schema=schema)
df_website_access.show()
df_website_access.printSchema()

In [ ]:
# 2.1 - Identifique as 10 páginas mais visitadas.

df_website_access.groupby("page_url")  \
    .count()                 \
    .sort(f.col("count")
          .desc())              \
    .show(10)

In [ ]:
# 2.2 Calcule a média de duração das sessões dos usuários.

df_website_access.select(f.avg("session_duration").alias("Duração média das sessões")) \
    .show(10)

In [ ]:
# 2.3 Determine quantos usuários retornam ao site mais de uma vez por semana.

# Cria uma coluna que identifica a semana e o ano
df_website_access = df_website_access.withColumn("year_week",f.concat_ws("-",f.year("date"),f.weekofyear("date")))

# Cria um Dataframe com os usuários que retornam com mais de 1 acesso semanal
df_weekly_website_access = df_website_access.groupBy("user_id","year_week").count().filter("count > 1")

df_returning_users = df_weekly_website_access.select("user_id").distinct().count()
output_message = f"{df_returning_users} usuários retornam ao site mais de uma vez por semana."
print(output_message)


### 3. Desenho de Arquitetura

#### Descrição

Proponha uma arquitetura em AWS para coletar dados de diferentes fontes:

- Desenhe um sistema para coletar dados de uma API.
- Processe esses dados em tempo real.
- Armazene os dados para análise futura.

### 4. Codificação

#### Descrição

Escreva um script Python para:

- Se conectar à [API de previsão do tempo OpenWeatherMap](https://openweathermap.org/api).
- Coletar dados dessa API para uma cidade de sua escolha.
- Armazenar os dados coletados em um banco de dados relacional ou NoSQL(de sua escolha).

In [ ]:
API_KEY = getenv("OPEN_WHEATER_API_KEY")
cidade = "miami"
link = f"https://api.openweathermap.org/data/2.5/weather?q={cidade}&appid={API_KEY}&lang=pt_br"

with req.get(link) as r:
    if r.status_code != 200:
        print("Erro ao fazer requisição: ",r.status_code)
    response = json.loads(r.content)

# descricao = response_dic['weather'][0]['description']
# temperatura = response_dic['main']['temp'] - 273.15
# print(descricao, f"{temperatura}ºC")